In [8]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import os
import time
from datetime import date, timedelta
from ipywidgets import *
from  bqwidgets import AutoComplete
import ipywidgets
import bqplot
from bqplot import ColorScale
import importlib
import multiple_select

importlib.reload(multiple_select) 

bq = bql.Service()
d = bq.data
f = bq.func

curr_widgets_version = -1

In [9]:
def build_accordion(name, width, margin, cb_func):
    accordion = Accordion()
    accordion.set_title(0, name)
    accordion.layout.width = '10px'

    if curr_widgets_version >= 7.3:
        accordion.selected_index = None
    else:
        accordion.selected_index = -1
    
    ms = multiple_select.MultipleSelect(width, margin, cb_func)
    accordion.children = [ms.panel]
    return accordion, ms

# rank dropdown
def accordion_call_back(tgt = None):
    
    country_list = accordion_countries_ms.read_values()
    if len(country_list) == 0:
        country_list = None
        
    issue_t_list = accordion_issue_type_ms .read_values()
    if len(issue_t_list) == 0:
        issue_t_list = None
        
    ratings_list = accordion_ratings_ms.read_values()
    if len(ratings_list) == 0:
        ratings_list = None
    
    sector_list = accordion_sector_ms.read_values()
    if len(sector_list) == 0:
        sector_list = None
        
# ************************************************************
#    Please wait message
# ************************************************************

wait_msg = HTML("""<h2 style="color:white;">Downloading data .....</h>
    <i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:white;"></i>""")
wait_msg.layout.height = '1px'
wait_msg.layout.visibility = 'hidden'
wait_msg.layout.margin = '20px 10px 10px 10px'        

In [10]:
#GUI    ********************

aux_label1 = Label(value='')
aux_label1.layout.height = '3px'

aux_label2 = Label(value='')
aux_label2.layout.width = '10px'

rank_YldorASW = Dropdown(options = ['Yield', 'ASW'])
rank_YldorASW.layout.width = '100px'

accordion_sector, accordion_sector_ms = build_accordion('Sectores', '180px', '1px', accordion_call_back)
accordion_countries, accordion_countries_ms = build_accordion('País', '180px', '1px', accordion_call_back)
accordion_ratings, accordion_ratings_ms = build_accordion('Rating', '180px', '1px', accordion_call_back)
accordion_issue_type, accordion_issue_type_ms = build_accordion('Tipo', '180px', '1px', accordion_call_back)

request_button1 = Button(description = 'AÑADIR', button_style = 'success')
#request_button1.on_click()
request_button2 = Button(description = 'BORRAR', button_style = 'danger')
#request_button2.on_click()

buttons = HBox([request_button1, request_button2])

inputs_top_row0 = VBox([rank_YldorASW, aux_label1, accordion_sector, aux_label1, accordion_countries, aux_label1, accordion_ratings, aux_label1, accordion_issue_type, aux_label1, buttons])

#Issuer list selection
label_select_issuers = Label(value='Características de las curvas')
select_issuers = SelectMultiple(options = [], rows=10)
select_issuers.layout.width = '100%'
select_issuers.layout.height = '90%'
#select_issuers.observe(issuer_select_event_handler, names = 'value')

inputs_top_row = HBox([inputs_top_row0, aux_label2, VBox([label_select_issuers, select_issuers])])


# ************************************************************
#   Chart
# ************************************************************

#Defining the axis types.
sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()
sc_col = ColorScale(colors=['Red', 'Green'])

#Defining the chart type.
line_sector = bqplot.Lines(x = [], y = [], scales={'x': sc_x, 'y': sc_y}, display_legend=True, colors=['orange'], marker = 'circle')

#Defining axis parameters. 
#num_ticks is useful to reduce clutter in the axis, 
ax_x = bqplot.Axis(scale = sc_x, label = 'Vencimientos')
ax_y = bqplot.Axis(scale=sc_y, orientation = 'vertical', tick_format = '0.2f', label = rank_YldorASW.value)

fig_line = bqplot.Figure(marks = [line_sector], axes = [ax_x, ax_y], legend_location = 'bottom')
fig_line.fig_margin = dict(top = 5, bottom = 20, left = 50, right = 5)
fig_line.layout.width = '100%'


#Term selection slider
min_tenor = 0.5
max_tenor = 35
tenor_select = IntRangeSlider(value = [min_tenor, max_tenor],  min = min_tenor, max = max_tenor)
tenor_select.layout.width = '100%'
# tenor_select.observe(tenor_slider_callback_handler, names = 'value')

output_box = VBox([wait_msg, fig_line, tenor_select])

main = VBox([inputs_top_row, output_box])
main

TraitError: Invalid selection: index out of bounds